In [1]:
import json

In [2]:
# Read words in to a dictionary
file = open('AFINN.txt')
word_dict = {}
for line in file.readlines():
    pair = line.split("\t")
    word_dict[pair[0]] = pair[1]

In [10]:
# Load melbGrid
location_list = [] # Store grid information
with open('melbGrid.json', 'r') as f:
    grid_dic = json.load(f)
    for location_dict in grid_dic['features']:
        location_list.append(location_dict['properties'])

In [23]:
# Given coordinate and grid information return a location id
def get_id(coordinate, location_list):
    x = coordinate[0]
    y = coordinate[1]
    for location in location_list:
        if x > location['xmin'] and x <= location['xmax'] \
            and y > location['ymin'] and y <= location['ymax']:
            return location['id']
    return False
print(get_id([145, -37.65], location_list))

B2


In [21]:
# Load coordinate and text
twitter_list = [] # Only sotre tweet in the specified location
with open('tinyTwitter.json', 'r', encoding = 'utf-8') as f:
    twitter_dic = json.load(f)

for row in twitter_dic['rows']:
    coordinate = row['value']['geometry']['coordinates']
    id = get_id(coordinate, location_list)
    if id:
        text = row['value']['properties']['text']
        twitter_list.append((id, text))
print(twitter_list[0])

('C2', "I'm at @RivaStkilda in St Kilda, VIC https://t.co/kEbicNcmnP")
